In [33]:
start_date = datetime.datetime(2023, 7, 1)
end_date = datetime.datetime(2024, 11, 1)

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

In [34]:
import yfinance as yf

# Scarica i dati per l'indice S&P 500
sp500 = yf.Ticker("^GSPC")

# Dati storici dell'S&P 500 (per esempio, degli ultimi 5 anni)
#data = sp500.history(period="max", interval="1m")
data = yf.download("^GSPC", start=start_date_str, end=end_date_str, interval="1mo")

print(data)

[*********************100%***********************]  1 of 1 completed

Price                        Adj Close        Close         High          Low  \
Ticker                           ^GSPC        ^GSPC        ^GSPC        ^GSPC   
Date                                                                            
2023-07-01 00:00:00+00:00  4588.959961  4588.959961  4607.069824  4385.049805   
2023-08-01 00:00:00+00:00  4507.660156  4507.660156  4584.620117  4335.310059   
2023-09-01 00:00:00+00:00  4288.049805  4288.049805  4541.250000  4238.629883   
2023-10-01 00:00:00+00:00  4193.799805  4193.799805  4393.569824  4103.779785   
2023-11-01 00:00:00+00:00  4567.799805  4567.799805  4587.640137  4197.740234   
2023-12-01 00:00:00+00:00  4769.830078  4769.830078  4793.299805  4546.500000   
2024-01-01 00:00:00+00:00  4845.649902  4845.649902  4931.089844  4682.109863   
2024-02-01 00:00:00+00:00  5096.270020  5096.270020  5111.060059  4853.520020   
2024-03-01 00:00:00+00:00  5254.350098  5254.350098  5264.850098  5056.819824   
2024-04-01 00:00:00+00:00  5

In [ ]:
import pandas as pd
import datetime as datetime

url_raw = "https://yfiua.github.io/index-constituents/$YYYY/$MM/constituents-sp500.csv"

# calcolo i costituentu dell'indice S&P 500 mese per mese a partire dal 2023/07 fino al 2024/10

stock_dict = {}

for date in pd.date_range(start_date, end_date, freq='MS'):
    print(date)
    url = url_raw.replace("$YYYY", str(date.year)).replace("$MM", str(date.month).zfill(2))
    print(url)
    # scarico il file CSV
    constituents = pd.read_csv(url)
    # aggiungo i dati storici
    print(constituents)
    for symbol in constituents['Symbol']:
        print(symbol)
        # Scarica i dati per il simbolo e salvali in un file CSV
        symbol_data = yf.download(symbol, start=start_date_str, end=end_date_str, interval="1mo")
        #print(symbol_data)
        if symbol not in stock_dict:
            stock_dict[symbol] = symbol_data
            #symbol_data.to_csv("stocks/" + symbol + ".csv")
        

    

2023-07-01 00:00:00
https://yfiua.github.io/index-constituents/2023/07/constituents-sp500.csv
    Symbol                   Name
0     AAPL              Apple Inc
1     MSFT         Microsoft Corp
2     AMZN         Amazon.com Inc
3     NVDA            Nvidia Corp
4    GOOGL      Alphabet Inc Cl A
..     ...                    ...
499    FOX     Fox Corp   Class B
500    LNC  Lincoln National Corp
501    NWL      Newell Brands Inc
502    NWS    News Corp   Class B
503   FTRE   Fortrea Holdings Inc

[504 rows x 2 columns]
AAPL


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

MSFT
AMZN



[*********************100%***********************]  1 of 1 completed


NVDA
GOOGL


...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


PSX
DXCM


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


FTNT
MCHP


[*********************100%***********************]  1 of 1 completed